In [1]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

In [2]:
device = torch.device("mps")
print(device)
#mac m1

mps


In [3]:
base_path = '/Users/nat/Documents/GitHub/BoneMonitor/datasets/images'

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
dataset = datasets.ImageFolder(root=base_path, transform=transform)

In [6]:
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [7]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


#preprocessing dataset done

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 28 * 28, 500)
        self.fc2 = nn.Linear(500, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 64 * 28 * 28)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize the CNN
model = CNN()
model.to(device)

print(model)


CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [9]:
import torch.optim as optim

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.0001)

# Number of epochs
n_epochs = 20

# Track changes in accuracy and loss
train_losses, test_losses, train_accuracy, test_accuracy = [], [], [], []

for epoch in range(1, n_epochs+1):
    train_loss = 0.0
    correct = 0
    total = 0

    # Training the model
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

    train_losses.append(train_loss/len(train_loader.dataset))
    train_accuracy.append(100 * correct / total)

    test_loss = 0.0
    correct = 0
    total = 0

    # Validating the model
    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item() * data.size(0)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    test_losses.append(test_loss/len(test_loader.dataset))
    test_accuracy.append(100 * correct / total)

    print(f'Epoch: {epoch}/{n_epochs} \tTraining Loss: {train_losses[-1]:.6f} \tValidation Loss: {test_losses[-1]:.6f} \tTraining Accuracy: {train_accuracy[-1]:.2f}% \tValidation Accuracy: {test_accuracy[-1]:.2f}%')


Epoch: 1/20 	Training Loss: 0.641987 	Validation Loss: 0.566071 	Training Accuracy: 62.78% 	Validation Accuracy: 72.00%
Epoch: 2/20 	Training Loss: 0.555499 	Validation Loss: 0.519354 	Training Accuracy: 71.15% 	Validation Accuracy: 76.00%
Epoch: 3/20 	Training Loss: 0.497224 	Validation Loss: 0.528550 	Training Accuracy: 77.73% 	Validation Accuracy: 73.33%
Epoch: 4/20 	Training Loss: 0.480880 	Validation Loss: 0.509732 	Training Accuracy: 75.78% 	Validation Accuracy: 74.67%
Epoch: 5/20 	Training Loss: 0.465401 	Validation Loss: 0.508281 	Training Accuracy: 77.13% 	Validation Accuracy: 77.33%
Epoch: 6/20 	Training Loss: 0.453705 	Validation Loss: 0.497525 	Training Accuracy: 78.18% 	Validation Accuracy: 74.67%
Epoch: 7/20 	Training Loss: 0.436232 	Validation Loss: 0.502613 	Training Accuracy: 79.67% 	Validation Accuracy: 73.33%
Epoch: 8/20 	Training Loss: 0.426372 	Validation Loss: 0.497493 	Training Accuracy: 80.42% 	Validation Accuracy: 77.33%
Epoch: 9/20 	Training Loss: 0.404695 	Va

In [10]:
model_dir = '/Users/nat/Documents/GitHub/BoneMonitor/models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [12]:
model_path = os.path.join(model_dir, 'classifier.pth')

# Save the model state
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to /Users/nat/Documents/GitHub/BoneMonitor/models/classifier.pth


In [14]:
from torchvision import datasets

# Assume your dataset is loaded using ImageFolder
dataset = datasets.ImageFolder(root=base_path)

# Print class to index mapping
print(dataset.class_to_idx)

{'normal': 0, 'osteoporosis': 1}
